<a href="https://colab.research.google.com/github/Imran-Github22/Programming-For-Data/blob/main/Projects/15_2_Decision_Tree_coded_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Coding a simple decision tree
---

In this worksheet we are going to work with a data set, using the idea of a decision tree class.  We are going to simplify the model and use Python code to make a simple decision tree classification model.  We will do this for two reasons:
*   writing the code is often good for helping to understand what is going on under the bonnet of a library function
*   it is a good coding exercise for practice as it mostly depends on calculations and if..elif..else statements

In this worksheet we are going to code a decision tree which will use the calculated probabilities to make decisions about whether a row of given data would be classified as Iris-virginica, or not, based on sepal and petal dimensions.  It is easier to classify between two values (Iris-virginica or not).  Later, using this information, species would be further predicted by probabilities of error.

![Iris-petals and sepals](https://www.math.umd.edu/~petersd/666/html/iris_with_labels.jpg)

The workflow is:
*  divide the data set into 70% of the rows for training and 30% for testing  (we can increase the size of the training set later)
*  find the mean for each of the 4 size columns
*  calculate the proportion of each column that are on or above mean that are of a species (ie proportion of petal-lengths on or above mean that are Iris-virginica)
*  infer the proportion of each that are not of that species (using 1 - proportion above).  In both cases we are looking to find if either of these is 1, which could be infered as definitely not that species. 
*  calculate a Gini Index that will indicate the probability that a prediction will be incorrect
*  use the results of the Gini Index to model a decision tree
*  code the decision tree model into a function that will return whether or not a row in the test set is predicted to be of species Iris-virginica
*  use the decision tree function to predict, for each row in the test set, if the species will be Iris-virginicia or not, using a set of nested if statements to classify
*  compare the predicted values against the actual values in the test set - what proportion were predicted correctly?


### Exercise 1 - investigate the iris data set
---
Let's start by looking at the data.  We are going to use a data set that contains data on iris flowers.

Read the data at this location: https://raw.githubusercontent.com/jbrownlee/Datasets/master/iris.csv into a dataframe called iris_data

The columns in the CSV file do not have headings, when you read the file, add column headings like this:
```
url = "https://raw.githubusercontent.com/jbrownlee/Datasets/master/iris.csv"
names = ['sepal-length', 'sepal-width', 'petal-length', 'petal-width', 'species']
iris_data = pd.read_csv(url, name=names)
```
*  Take a look at the column info (how many columns, what type of data, any missing data?)
*  Take a look at the data values in the first 10 and the last 10 records to get an idea of the type of values included
*  Find out how many unique values there are in the species column
*  Find out the maximum, minimum, mean, median and upper and lower quartile values in each of the columns


In [1]:
import pandas as pd
import numpy as np

url = "https://raw.githubusercontent.com/jbrownlee/Datasets/master/iris.csv"
names = ['sepal-length', 'sepal-width', 'petal-length', 'petal-width', 'species']
iris_data = pd.read_csv(url, names=names)

# Task 1
display(iris_data.info())
print('')

# Task 2
display(iris_data.head(10))
print('')
display(iris_data.tail(10))
print('')

#Task 3
unique_val_species = iris_data['species'].unique()
print(f'Unique Species# {len(unique_val_species)}.\n')
print(unique_val_species)
#Task 4
iris_describe_df = iris_data.describe()
iris_median_dict = dict(iris_data[['sepal-length', 'sepal-width', 'petal-length', 'petal-width']].median(), index=['median'])
iris_stats_df = pd.DataFrame.from_records(iris_median_dict, index=['median'])
iris_stats_df.drop(columns=['index'], inplace=True)
iris_stats_df = pd.concat([iris_describe_df, iris_stats_df])
display(iris_stats_df)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   sepal-length  150 non-null    float64
 1   sepal-width   150 non-null    float64
 2   petal-length  150 non-null    float64
 3   petal-width   150 non-null    float64
 4   species       150 non-null    object 
dtypes: float64(4), object(1)
memory usage: 6.0+ KB


None

,sepal-length,sepal-width,petal-length,petal-width,species
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa
5,5.4,3.9,1.7,0.4,Iris-setosa
6,4.6,3.4,1.4,0.3,Iris-setosa
7,5.0,3.4,1.5,0.2,Iris-setosa
8,4.4,2.9,1.4,0.2,Iris-setosa
9,4.9,3.1,1.5,0.1,Iris-setosa


,sepal-length,sepal-width,petal-length,petal-width,species
140,6.7,3.1,5.6,2.4,Iris-virginica
141,6.9,3.1,5.1,2.3,Iris-virginica
142,5.8,2.7,5.1,1.9,Iris-virginica
143,6.8,3.2,5.9,2.3,Iris-virginica
144,6.7,3.3,5.7,2.5,Iris-virginica
145,6.7,3.0,5.2,2.3,Iris-virginica
146,6.3,2.5,5.0,1.9,Iris-virginica
147,6.5,3.0,5.2,2.0,Iris-virginica
148,6.2,3.4,5.4,2.3,Iris-virginica
149,5.9,3.0,5.1,1.8,Iris-virginica



Unique Species# 3.

['Iris-setosa' 'Iris-versicolor' 'Iris-virginica']


,sepal-length,sepal-width,petal-length,petal-width
count,150.000000,150.000000,150.000000,150.000000
mean,5.843333,3.054000,3.758667,1.198667
std,0.828066,0.433594,1.764420,0.763161
min,4.300000,2.000000,1.000000,0.100000
25%,5.100000,2.800000,1.600000,0.300000
50%,5.800000,3.000000,4.350000,1.300000
75%,6.400000,3.300000,5.100000,1.800000
max,7.900000,4.400000,6.900000,2.500000
median,5.800000,3.000000,4.350000,1.300000


### Exercise 2 - split the data into train and test sets
---

Split the data set into and 70% train, 30% test, split.  From now, just use the train data set.


In [15]:
# import the train_test_split function
from sklearn.model_selection import train_test_split

option = int(input('For training set to use: "mean", "median", "upper-quartile" \
or "lower-quartile",\nPlease input either option: 0, 1, 2, or 3? '))
option_txt = ['mean', 'median', 'upper-quartile', 'lower-quartile']
training_set = option_txt[option]
print(f"The dataset splitted into 'train' and 'test' for the: {training_set}\n")

train, test = train_test_split(iris_data, test_size=0.30)

For training set to use: "mean", "median", "upper-quartile" or "lower-quartile",
Please input either option: 0, 1, 2, or 3? 0
The dataset splitted into 'train' and 'test' for the: mean



### Exercise 3 - assumptions and classification
---

Let's make some assumptions based on the data

1.  Iris-setosa, Iris-versicolor, Iris-virginica are the full range of types of iris to be analysed
2.  Although this is a small data set, the means are fairly representative

With these in mind, let's start by classifying sepal/petal size into long/short and wide/narrow with values on or above the mean taken as long or wide and those below as short or narrow.

This is a starting point.  We will be trying to find a value (indicator) for each column where rows on or above do not contain any of a particular species, this might indicate that this column is a good (if not rough) indicator of species.  For now, the indicator is the mean.

*  Drop any null values from each column

Calculate, and store the means of the four columns

*  **Test**:
Display train.describe() to see the value of the means of the training set. Print the four means and compare to the output of train.describe() to check that they have been calculated correctly.

*  Create a new dataframe with the numeric columns encoded so show a 1 for any value that is above the mean for its column and 0 for any that isn't.




In [3]:
# get the mean for each column and apply a function to encode into 1 (above mean) and 0 (mean or below mean)
train.dropna(inplace=True)

if training_set == 'mean':
  print(f'{training_set}:', round(train['sepal-length'].mean(), 6), \
        round(train['sepal-width'].mean(), 6), \
        round(train['petal-length'].mean(), 6), \
        round(train['petal-width'].mean(), 6))
elif training_set == 'median':
  print(f'{training_set}:', round(train['sepal-length'].median(), 6), \
        round(train['sepal-width'].median(), 6), \
        round(train['petal-length'].median(), 6), \
        round(train['petal-width'].median(), 6))
elif training_set == 'upper-quartile':
  print(f'{training_set}:', round(train['sepal-length'].quantile(0.75), 6), \
        round(train['sepal-width'].quantile(0.75), 6), \
        round(train['petal-length'].quantile(0.75), 6), \
        round(train['petal-width'].quantile(0.75), 6))
elif training_set == 'lower-quartile':
  print(f'{training_set}:', round(train['sepal-length'].quantile(0.25), 6), \
        round(train['sepal-width'].quantile(0.25), 6), \
        round(train['petal-length'].quantile(0.25), 6), \
        round(train['petal-width'].quantile(0.25), 6))
  
display(train.describe())

mean: 5.930476 3.090476 3.859048 1.225714


,sepal-length,sepal-width,petal-length,petal-width
count,105.000000,105.000000,105.000000,105.000000
mean,5.930476,3.090476,3.859048,1.225714
std,0.815018,0.406799,1.749654,0.756005
min,4.400000,2.200000,1.000000,0.100000
25%,5.200000,2.800000,1.600000,0.300000
50%,5.900000,3.000000,4.400000,1.400000
75%,6.500000,3.300000,5.100000,1.800000
max,7.900000,4.400000,6.900000,2.500000


In [4]:
def encode(df, **kwds):
  key = kwds['key']
  indicator = kwds['indicator']
  if df[key] >= indicator:
    return 1
  else:
    return 0
  return

# run the function for each column so that each of the four columns are encoded,
# then drop the original columns, saving as a new dataframe
def get_encoded_df(df, columns, training_set):
  if training_set == 'mean':
    for column in columns:
      df[column + '-' + training_set + '-encoded'] = \
      df.apply(encode, axis=1, key=column, indicator=df[column].mean())
    df.drop(columns=columns, inplace=True)
  elif training_set == 'median':
    for column in columns:
      df[column +'-' +  training_set + '-encoded'] = \
      df.apply(encode, axis=1, key=column, indicator=df[column].median())
    df.drop(columns=columns, inplace=True)
  elif training_set == 'upper-quartile':
    for column in columns:
      df[column +'-' +  training_set + '-encoded'] = \
      df.apply(encode, axis=1, key=column, indicator=df[column].quantile(0.75))
    df.drop(columns=columns, inplace=True)
  elif training_set == 'lower-quartile':
    for column in columns:
      df[column +'-' +  training_set + '-encoded'] = \
      df.apply(encode, axis=1, key=column, indicator=df[column].quantile(0.25))
    df.drop(columns=columns, inplace=True)
  else:
      print('Sorry, yet to programme it :) ')  
  return df

numaral_columns = names[:-1]

train_encoded_df = get_encoded_df(train, numaral_columns, training_set)
encoded_columns = train_encoded_df.columns[1:]
display(train_encoded_df.describe())

,sepal-length-mean-encoded,sepal-width-mean-encoded,petal-length-mean-encoded,petal-width-mean-encoded
count,105.000000,105.000000,105.000000,105.000000
mean,0.485714,0.466667,0.657143,0.609524
std,0.502193,0.501280,0.476941,0.490197
min,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,1.000000,1.000000
75%,1.000000,1.000000,1.000000,1.000000
max,1.000000,1.000000,1.000000,1.000000


### Exercise 4 - Calculate the proportion of values on or above the mean that are of each species

We are going to focus on the `Iris-virginica` species first.

First we will calculate, for each dimension column (`sepal-length, sepal-width, petal-length, petal-width`) what proportion of values in that column, where the value is on or above the mean, are classified as `Iris-virginica`.

We will do this by filtering all the records in each column of the the `train` set that are on or above the mean and match the species .  Then use the outcome to calculate the proportion of the full `train` set for which a value on or above the mean that are of species `Iris-virginica`.

*  filter for values in the `sepal-length` column being on or above the mean and the species column being `Iris-virginica`.  Then divide the count of rows in this filtered dataset by the count of rows in a second data set, filtered for just the value being on or above the mean.

*  Do this for all four columns, for `Iris-virginica`  (4 operations).

Print the results to see which columns look like they might most reliably predict the species as `Iris-virginica` (the result is as close as possible to 1).  The highest numbers may indicate the most reliable indicators, but we will do some more before coming to this conclusion.

*  By definition, those on or above the mean that are NOT Iris_virginica will be `1 - the proportion of those that are.  Calculate these

The first one has been done for you.

*  We will also need the proportion of those BELOW the mean that are NOT Iris-virginica.  Calculate these in the same way



In [5]:
# Debug
# display(train[train['species']=='Iris-virginica'])
# display(train[train['species']=='Iris-virginica'].describe())
# display(train[train['species']=='Iris-setosa'])
# display(train[train['species']=='Iris-setosa'].describe())
# display(train[train['species']=='Iris-versicolor'])
# display(train[train['species']=='Iris-versicolor'].describe())  

In [6]:
# calculate the proportion of results where the value is on or above mean 
# that are of the species Iris-virginica
def get_proportion(df, columns, species, above_equal_indicator = True):
  if above_equal_indicator:
    Flag = 1
    index_str = 'Above/Equal Indicator:'
  else:
    Flag = 0  
    index_str = 'Below Indicator:'
  
  # Species only DataFrame
  species_df = df[df['species']==species]

  # Get proportions
  # species_Flag_dict dictionary with counts for the specific 'species'
  # species_Flag_dict = dict(species_df[encoded_columns].sum(), index=index_str)

  species_Flag_dict = {}
  species_Flag_dict['index'] = index_str
  for column in columns:
    species_Flag_dict[column] = len(species_df[species_df[column] == Flag])
  species_Flag_dict = dict(species_Flag_dict)

  species_prop_dict = {}
  species_prop_dict['index'] = 'proportion:'
  for column in columns:
    species_prop_dict[column] = species_Flag_dict[column]/(df[column].sum())
  species_prop_dict = dict(species_prop_dict)

  species_except_dict = {}
  species_except_dict['index'] = '(1 - proportion)'
  for column in columns:
    species_except_dict[column] = 1 - species_prop_dict[column]
  species_except_dict = dict(species_except_dict)

  species_Flag_df = pd.DataFrame.from_dict([species_Flag_dict, species_prop_dict, species_except_dict])
  return species_Flag_df

iris_virginica_eq_df = get_proportion(train_encoded_df, encoded_columns, 'Iris-virginica')
display(iris_virginica_eq_df)

,index,sepal-length-mean-encoded,sepal-width-mean-encoded,petal-length-mean-encoded,petal-width-mean-encoded
0,Above/Equal Indicator:,32.000000,11.00000,36.000000,36.0000
1,proportion:,0.627451,0.22449,0.521739,0.5625
2,(1 - proportion),0.372549,0.77551,0.478261,0.4375


### Exercise 5 - Calculate the proportion of each column, where the value is below mean, that are of species `Iris-virginica`

Repeat the code above, this time looking for values below the mean

In [7]:
# calculate the proportion of results where the value is below mean that are of the species Iris-virginica
iris_virginica_bw_df = get_proportion(train_encoded_df, encoded_columns, 'Iris-virginica', False)
display(iris_virginica_bw_df)

,index,sepal-length-mean-encoded,sepal-width-mean-encoded,petal-length-mean-encoded,petal-width-mean-encoded
0,Below Indicator:,4.000000,25.000000,0.0,0.0
1,proportion:,0.078431,0.510204,0.0,0.0
2,(1 - proportion),0.921569,0.489796,1.0,1.0


### Exercise 5 - calculate for the other two Iris species
---

Do the same calculations for the Iris-versicolor species, then for the Iris-setosa species.






In [8]:
# calculate the proportion of results where the value is on or above mean that are of the species Iris-versicolor
iris_versic_eq_df = get_proportion(train_encoded_df, encoded_columns, 'Iris-versicolor')
display(iris_versic_eq_df)

,index,sepal-length-mean-encoded,sepal-width-mean-encoded,petal-length-mean-encoded,petal-width-mean-encoded
0,Above/Equal Indicator:,19.000000,8.000000,33.000000,28.0000
1,proportion:,0.372549,0.163265,0.478261,0.4375
2,(1 - proportion),0.627451,0.836735,0.521739,0.5625


In [9]:
# calculate the proportion of results where the value is below mean that are of the species Iris-versicolor
iris_versic_bw_df = get_proportion(train_encoded_df, encoded_columns, 'Iris-versicolor', False)
display(iris_versic_bw_df)

,index,sepal-length-mean-encoded,sepal-width-mean-encoded,petal-length-mean-encoded,petal-width-mean-encoded
0,Below Indicator:,17.000000,28.000000,3.000000,8.000
1,proportion:,0.333333,0.571429,0.043478,0.125
2,(1 - proportion),0.666667,0.428571,0.956522,0.875


In [10]:
# calculate the proportion of results where the value is on or above mean that are of the species Iris-setosa
iris_setosa_eq_df = get_proportion(train_encoded_df, encoded_columns, 'Iris-setosa')
display(iris_setosa_eq_df)

,index,sepal-length-mean-encoded,sepal-width-mean-encoded,petal-length-mean-encoded,petal-width-mean-encoded
0,Above/Equal Indicator:,0.0,30.000000,0.0,0.0
1,proportion:,0.0,0.612245,0.0,0.0
2,(1 - proportion),1.0,0.387755,1.0,1.0


In [11]:
# calculate the proportion of results where the value is below mean that are of the species Iris-setosa
iris_setosa_bw_df = get_proportion(train_encoded_df, encoded_columns, 'Iris-setosa', False)
display(iris_setosa_bw_df)

,index,sepal-length-mean-encoded,sepal-width-mean-encoded,petal-length-mean-encoded,petal-width-mean-encoded
0,Below Indicator:,33.000000,3.000000,33.000000,33.000000
1,proportion:,0.647059,0.061224,0.478261,0.515625
2,(1 - proportion),0.352941,0.938776,0.521739,0.484375


### Exercise 6 - predict from the results
---

Create a list of dictionaries from the results Exercise 4 and 5 (e.g. {'species':..., 'above_mean': 0.xx, 'below_mean': 0.xx}  

Then use a loop to go through the list and print:  
*  any species and indicator (above or below mean) that can reliably be predicted.  A reliable prediction may be one over 0.5

In [12]:
# show which columns are reliable predictors



### Exercise 6 - Make a decision tree
---

Use pencil and paper or a graphical application to create a decision tree for Iris-virginica, using the following rules (use the picture below as a guide):

*  The column with the highest indicator is placed at the top
*  Other columns are placed in order below
*  The rest of the columns are placed in order below these

Any column where one branch (on or above mean OR below mean) has an indicator of 0, could be classified as a strong indicator of Iris_virginica being the species.  Anything else, unless there is something very close to 0, could be classified as a weak indicator of Iris_virginica being the species.

Let's code the decision tree using the following logic for this decision tree (yours might be slightly different):

![Decision tree](https://drive.google.com/uc?id=1CTo23EHwR2IPCRjcfSyCQsT_oQ5Exwso)

In the decision tree above, there is no certainty below petal-length so our decision tree will only include petal-width and petal-length.




In [13]:
def get_species(df):
  # ADD CODE HERE TO RETURN None if petal-width is below mean (encoded as 0) or if petal-length is below mean (encoded as 0), otherwise return 'Iris-virginia'

  return
# use the get_species(df) function to predict the species, count how many are predicted correct and use this to calculate the proportion correct
correct = 0
test_size = test.shape[0]
for i in range(0, test_size):
  species = get_species(test.iloc[i])
  if species == test.iloc[i]['species']:
      correct += 1

print ("Proportion correctly identified", correct / test_size) 


Proportion correctly identified 0.0


### Exercise 7 - change the measure

We are currently using the mean to act as the decision making line.  We can use the decision tree with a different line.

Change the mean values so that you are instead using the median instead for all four columns.  The code should not need changing except for where you calculated the mean.

Run all the code again.  Is the proportion of correct values better this time?   Is the decision tree still appropriate?


What do you notice? (write your answer here)

### Exercise 8 - try different measures
---

Do the same again but with upper quantile, then again with the lower quantile.  Is it making any difference?  Which give the best looking results?

### Exercise 9 - try a different species

Run the mean test again for the Iris-versicolor species.  Again, try some different decision making lines.

What are the results.  Record them in the text cell below:

Write your answers here:  

# New logic introduced in this worksheet:

1.  Adding headings to a CSV if none currently exist
2.  Splitting a data set into train and test sets

In [14]:
## this type of plot will show the distribution on a chart
from plotnine import *
ggplot(train, aes(x='petal-length', y='petal-width', color='species')) + geom_point() + geom_vline(train, aes(xintercept=train['sepal-length'].mean(), color='species')) + geom_hline(train, aes(yintercept=train['sepal-length'].mean(), color='species'))

KeyError: ignored